In [29]:
import torch
import pandas as pd
import numpy as np

# Import dataset utils
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import importlib
if importlib.util.find_spec('ipywidgets') is not None:
    from tqdm.auto import tqdm
else:
    from tqdm import tqdm

In [30]:
dataframe = pd.read_csv('../data/final.csv', sep=';')
dataframe.head()

,sexo,Estado_civil,Status_empl,Licenca,Tipo_Resid,Residencia,Alcoolatra,Droga,Suic_familia,Dep_familia,...,Eixo I: Panico sem agorafobia,Eixo I: Fobia especifica,Eixo I: Fobia social,Eixo I: Obsessivo-compulsivo,Eixo I: Estresse pos-traumatico,Eixo I: Ansiedade generalizada,Eixo II: Personalidade paranoica,Eixo II: Transtorno de personalidade,TOC,idade
0,M,3.0,NaN,0.0,3.0,1.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,40.0
1,F,1.0,3.0,0.0,4.0,3.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0
2,F,1.0,2.0,0.0,1.0,2.0,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
3,F,1.0,3.0,0.0,1.0,3.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,30.0
4,F,4.0,2.0,0.0,1.0,NaN,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0


In [31]:
selected = ['Suicidio', 'Trabalho e interesses', 'Apetite', 'Sentimentos_culpa', 'Perda de insights',
            'Ansiedade somática', 'Ansiedade', 'Perda de peso', 'Lentidao pensamento e fala',
            'Hipocondriase', 'Energia', 'Libido',  'sexo','Pontuação total', 'Deprimido']

df_suic = dataframe[selected]

df_suic['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
df_suic['sexo'].fillna(0, inplace=True)

df_suic.dropna(inplace=True)
df_suic = df_suic.astype(int)

/tmp/ipykernel_499340/455746505.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suic['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
/tmp/ipykernel_499340/455746505.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suic['sexo'].fillna(0, inplace=True)
/tmp/ipykernel_499340/455746505.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suic.dropna(inplace=True)


In [32]:
class MyDataset(Dataset):
 
  def __init__(self, input_dataframe, split="train", target="Suicidio", ignore_columns=[], train_ratio=0.8):
    
    self.split = split
    self.target = target
    self.ignore_columns = ignore_columns

    for coll in self.ignore_columns:
       if coll in input_dataframe.columns:
        input_dataframe = input_dataframe.drop(coll, axis=1)

    self.classification_dim = len(input_dataframe[self.target].unique())
    self.data_dim = len(input_dataframe.columns) - 1
    self.embbeding_dim = input_dataframe.max().max() + 1

    y = input_dataframe[target].values
    x = input_dataframe.drop(target, axis = 1).values

    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=42)

  def __len__(self):
    if self.split == "train":
      return len(self.x_train)
    elif self.split == "test":
      return len(self.x_test)
    else:
      raise ValueError("Split must be train or test")

  def __getitem__(self,idx):
    target = torch.zeros(self.classification_dim)

    if self.split == "train":
      target[self.y_train[idx]] = 1
      return (torch.tensor(self.x_train[idx]), target)
    elif self.split == "test":
      target[self.y_test[idx]] = 1
      return (torch.tensor(self.x_test[idx]), target)
    else:
      raise ValueError("Split must be train or test")


In [33]:
train_dataset = MyDataset(df_suic, split="train", target="Suicidio", ignore_columns=[], train_ratio=0.8)
test_dataset = MyDataset(df_suic, split="test", target="Suicidio", ignore_columns=[], train_ratio=0.8)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [34]:
## Define a MLP model with N layers

import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, n_layers=2):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
    
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(self.input_dim, self.hidden_dim))
        for i in range(self.n_layers - 1):
            self.layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            self.layers.append(nn.Dropout(0.5))
            # self.layers.append(nn.BatchNorm1d(self.hidden_dim))
            self.layers.append(nn.ReLU())

        self.layers.append(nn.Linear(self.hidden_dim, self.output_dim))
    
    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
        return x

# Define a Model with a embbeding layer and a MLP

class ClassificationModel(nn.Module):
    def __init__(self, input_dim, output_dim, embbeding_dim, hidden_out, hidden_dim=128, n_layers=2):
        super(ClassificationModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embbeding_dim = embbeding_dim
        self.embbeding_out = hidden_out
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embbeding_layer = nn.Embedding(self.embbeding_dim, self.embbeding_out)
        self.mlp = MLP(self.input_dim * self.embbeding_out, self.output_dim, self.hidden_dim, self.n_layers)

    def forward(self, x):
        x = self.embbeding_layer(x)
        x = x.view(x.shape[0], -1)
        x = self.mlp(x)
        ## classification
        x = F.softmax(x, dim=1)
        return x

# test the model
example_batch = next(iter(train_loader))
example_data, example_targets = example_batch

model = ClassificationModel(train_dataset.data_dim, train_dataset.classification_dim, train_dataset.embbeding_dim, hidden_out=20, hidden_dim=128, n_layers=4)
print(model)
print("Batch shape:", example_data.shape)
res = model(example_data)
print("Output shape:", res.shape)

ClassificationModel(
  (embbeding_layer): Embedding(44, 20)
  (mlp): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=280, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Dropout(p=0.5, inplace=False)
      (3): ReLU()
      (4): Linear(in_features=128, out_features=128, bias=True)
      (5): Dropout(p=0.5, inplace=False)
      (6): ReLU()
      (7): Linear(in_features=128, out_features=128, bias=True)
      (8): Dropout(p=0.5, inplace=False)
      (9): ReLU()
      (10): Linear(in_features=128, out_features=5, bias=True)
    )
  )
)
Batch shape: torch.Size([128, 14])
Output shape: torch.Size([128, 5])


In [50]:
## Make Lightning Module
from pytorch_lightning import LightningModule

class BaseModel(LightningModule):
    """A LightningModule organizes your PyTorch code into 6 sections:
        - Computations (init)
        - Validation loop (validation_step)
        - Train loop (training_step)
        - Test loop (test_step)
        - Prediction Loop (predict_step)
        - Optimizers and LR Schedulers (configure_optimizers)
    """

    def __init__(self, input_dim, output_dim, embedding_dim, embedding_out, hidden_dim):
        super().__init__()
        self.model = ClassificationModel(input_dim, output_dim, embedding_dim, embedding_out, hidden_dim=hidden_dim, n_layers=2)
        self.lr = 1e-3

        # save hyperparameters
        self.save_hyperparameters()

    def step(self, batch):
        x, y = batch
        y_hat = self.model(x).squeeze().float()
        loss = F.binary_cross_entropy(y_hat, y)
        # L1 Loss
        # loss = F.l1_loss(y_hat, y)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer


In [51]:
# Import trainer
from pytorch_lightning.trainer import Trainer


# Initialize model
model = BaseModel(input_dim=train_dataset.data_dim, output_dim=train_dataset.classification_dim, embedding_dim=100, embedding_out=64, hidden_dim=128)
print(model)

BaseModel(
  (model): ClassificationModel(
    (embbeding_layer): Embedding(100, 64)
    (mlp): MLP(
      (layers): ModuleList(
        (0): Linear(in_features=896, out_features=128, bias=True)
        (1): Linear(in_features=128, out_features=128, bias=True)
        (2): Dropout(p=0.5, inplace=False)
        (3): ReLU()
        (4): Linear(in_features=128, out_features=5, bias=True)
      )
    )
  )
)


In [52]:
# Import callbacks
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Initialize callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min',
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.05,
    patience=10,
    verbose=False,
    mode='min'
)

callbacks = [checkpoint_callback, early_stopping]
# callbacks = []


# Initialize a trainer
trainer = Trainer(accelerator='gpu', devices=1, check_val_every_n_epoch=10, log_every_n_steps=10, callbacks=callbacks, auto_lr_find=True, enable_progress_bar=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [53]:
# trainer.tune(model, train_loader)

In [54]:
# Train the model ⚡
trainer.fit(model, train_loader, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | ClassificationModel | 138 K 
----------------------------------------------
138 K     Trainable params
0         Non-trainable params
138 K     Total params
0.553     Total estimated model params size (MB)
